In [152]:
import vk_api
import json
from pandas.io.json import json_normalize
import pandas as pd
import datetime

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


Задача предсказать возраст

метрика MSE

In [153]:
vk_session = vk_api.VkApi('+77056735885', '211907098381aa')
vk_session.auth()



In [154]:
vk_api = vk_session.get_api()

In [155]:
#posts = getLikes (5117073,1, vk_api)

In [156]:
def get_friends(userId, api):
    user_friends = api.friends.get(user_id=userId, order='hints', fields = 'id, nickname, domain, sex, bdate, city, country, timezone, has_mobile, contacts, education, online, relation,  status, universities , faculty, faculty_name, relation') # возвращает в том порядке, как расположены в разделе Мои
    return user_friends

In [157]:
def get_posts(userId, api):
    user_posts = api.wall.get(owner_id=userId, filter='owner') 
    return user_posts

In [158]:
def transformDataToDf(data, file_txt, file_csv):
    with open(file_txt,'w', encoding='utf-8') as file:
        json.dump(data,file,ensure_ascii=False)
        
    df = pd.read_json(file_txt, encoding='utf-8')
    df2 = json_normalize(df['items'])
    df2.to_csv(file_csv,sep = ',')
    
    df = pd.read_csv(file_csv,  sep=',')
    
    return df

### Получение данных по друзьям

In [159]:
friends =  get_friends(5117073,vk_api)

In [160]:
df_f = transformDataToDf(friends, 'c:/friends.txt', 'c:/friends.csv')

In [161]:
df_f[df_f['deactivated']=='deleted'] 
df_f.shape
df_f = df_f.drop(df_f[df_f.deactivated == 'deleted'].index)
df_f.shape

(125, 35)

In [162]:
#Определяем возраст человека, если не указан ставим 0
df_f['bdate'] = df_f['bdate'].astype('str')
df_f['bdate']= df_f['bdate'].apply(lambda x: '01.01.2019' if len(x) < 8 else x)
df_f['age'] = df_f.bdate.str[-4:]
df_f['age'] = df_f['age'].astype('int')
tday = datetime.datetime.now()
df_f['age2'] = tday.year
df_f['age2'].astype('int')
df_f['age3'] = df_f['age2'] - df_f['age']




In [163]:
cols = ['id','first_name','last_name','graduation', 'city.id','city.title','country.id','country.title','education_form','education_status','faculty','faculty_name','relation','relation_partner.id',
'sex','status','bdate','age3']
df_f = df_f[cols]

In [40]:
df_f.head()

,id,first_name,last_name,graduation,city.id,city.title,country.id,country.title,education_form,education_status,faculty,faculty_name,relation,relation_partner.id,sex,status,bdate,age3
0,88111,Анастасия,Ахметзянова,0.0,2.0,Санкт-Петербург,1.0,Россия,NaN,NaN,0.0,NaN,3.0,NaN,1,All you need is love,01.01.2019,0
1,3567843,Константин,Стрельцов,2004.0,284.0,Караганда,4.0,Казахстан,Очное отделение,NaN,9918.0,Факультет инновационных технологий (Информацио...,4.0,NaN,2,NaN,19.7.1982,37
2,9067562,Даша,Баскакова,0.0,99.0,Новосибирск,1.0,Россия,NaN,NaN,0.0,NaN,4.0,1663884.0,1,NaN,01.01.2019,0
3,7862738,Ольга,Стрельцова,0.0,284.0,Караганда,4.0,Казахстан,NaN,NaN,0.0,NaN,4.0,NaN,1,Happiness is the truth,11.1.1983,36
4,202788920,Margo,Almatyfashion,0.0,183.0,Алматы,4.0,Казахстан,NaN,NaN,0.0,NaN,0.0,NaN,1,Тел 87078143309,01.01.2019,0


### Получение данных - посты
собираем в цикле посты каждого друга

In [17]:
users_list = df_f.id

In [18]:
#df = df[df['text'].notnull()]
#cols = ['owner_id','text','comments.count','date','post_source.platform','likes.count','reposts.count']
#df = df[cols]

In [ ]:
df_all = pd.DataFrame()
for user_id in users_list:
        print(user_id)
        posts =  get_posts(user_id,vk_api)
        df = transformDataToDf(posts, 'c:/data.txt', 'c:/data.csv')
        df = df[df['text'].notnull()]
        cols = ['owner_id','text','comments.count','date','likes.count','reposts.count']
        df = df[cols]
        df_all = df_all.append(df)
        

In [ ]:
#выгружаем в файл для повторного использования без обращения к вк
df_all.to_csv('c:/data_all.csv',sep = ',', encoding='utf-8')

In [122]:
#считаваем из файла, чтобы не обращатья повторно к вк (после первого прохода, запускаем только получение из файла)
df_a = pd.read_csv('c:/data_all.csv',  sep=',')

In [123]:
#объединяем данные с постов с информацией о пользователе
df_u = pd.merge(df_a,df_f, how = 'inner',left_on="owner_id" , right_on="id" )

In [124]:
df_val = df_u[df_u['age3']==0]
df_u = df_u[df_u['age3']>0]





### Обработка текста

In [164]:
text = df_u['text']

In [165]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string


stop = set(stopwords.words('russian'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()


def clean(doc):
    #print(doc)
    doc = str(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join([ch for ch in stop_free if ch not in exclude])
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized




In [166]:
doc_clean = [clean(doc) for doc in df_u['text']]   
print(doc_clean[149])

готовимся новогоднему выступлению


In [69]:
#import gensim
#from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
#dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
#doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [167]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(doc_clean).todense()
features

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [174]:
y = df_u['age3']
X = features

Попробуем линейную регрессию как baseline


In [175]:
from sklearn import cross_validation, tree, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
regr = linear_model.LinearRegression()

In [171]:
#df_u['tokens'] = df_u['text'].apply(lambda x: clean(x).split())


In [172]:
#df_u['vector'] = df_u['tokens'].apply(lambda x: dictionary.doc2bow(x))

In [ ]:
#cols = ['id', 'city.id','country.id','relation','sex','graduation']
#X = df_u[cols]
#y = df_u['age3']

In [176]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [177]:
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [178]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f " % mse)

MSE: 1568606792827209747791872.00 
